In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt
import transformers
import torch
from torch import nn
from torch.nn import functional as F
import re
import os
import random
import pickle
from label_processor import LabelProcessorSimplified


In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [8]:
df2 = pickle.load(open("data/df2.pickle","rb"))
labelizer = torch.load("/data/data/saved/labelizer.torch")

In [6]:
tqdm.pandas(desc="find tokens")
token_lambda = lambda a: re.findall("[\[|\{]+\w+[\]|\}]+",a) if type(a)==str else np.nan
a = df2.message.progress_apply(token_lambda)
a = a.progress_apply(lambda a:a if type(a) == list and len(a) > 0 else np.nan).dropna()
b = [i for j in a for i in j]
b = pd.Series(b).value_counts()

RuntimeError: No active exception to reraise

In [18]:
# Add extra tokens
def add_extra_tokens():
    from spacy.attrs import ORTH, NORM
    for i,j in b.items():
        if j>100:
            nlp.tokenizer.add_special_case(i,[{ORTH:i}])
add_extra_tokens()

In [27]:
def tokenize_to_ids(s):
    s = nlp(s)
    return [i.lex_id if i.lex_id!=0 else i.string for i in s]

In [28]:
import math
def chunks(l, n:int):
    n = math.ceil(l/n)
    def a():
        for i in range(0,l,n):
            yield (i, i+n)
    return list(a())

In [33]:
import os
upto = max([int(re.findall("chunk_(\d+)",i)[0]) \
            for i in os.listdir("saved/word2vec/tokenized_messages/") if "chunk" in i])

In [ ]:
for i,(start, end) in enumerate(tqdm(chunks(df2.message.shape[0],1000))):
    if i < upto:
        continue
    if i % 10 == 0:
        del nlp
        nlp = spacy.load("en_core_web_lg")
        add_extra_tokens()
    tokenized_messages = df2.message[start:end].apply(tokenize_to_ids)
    tokenized_messages.to_pickle(f"saved/word2vec/tokenized_messages/chunk_{i}.pickle")

In [37]:
tokenized_messages_list = [pd.read_pickle(f"saved/word2vec/tokenized_messages/chunk_{i}.pickle") for i in trange(1000)]
    

In [40]:
tokenized_messages.to_pickle("saved/word2vec/tokenized_messages_all")

In [395]:
import multiprocessing as mp
with mp.Pool(4) as pool:
    tokenized_messages = list(pool.imap(tokenize_to_ids, tqdm(df2.message)))
pickle.dump(tokenized_messages, open("saved/word2vec/spacy_tokenized_messages.pickle","wb"))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/daniel.cahn/ana

KeyboardInterrupt: 

In [4]:
tokenized_messages = pickle.load(open("saved/word2vec/spacy_tokenized_messages","rb"))

# PostProcessing

In [2]:
tokenized_messages = pd.read_pickle("saved/word2vec/tokenized_messages_all")

In [5]:
all_tokens = {i for j in tqdm(tokenized_messages) for i in j}

In [6]:
tokens = {i for i in all_tokens if type(i) == int}
tokens.add(0)
tokens = sorted(tokens)
d = {token_id:i for i,token_id in enumerate(tokens)}

In [7]:
potential_new_tokens = [i for j in tqdm(tokenized_messages) for i in j if type(i) != int]

In [8]:
vc = pd.Series(potential_new_tokens).str.strip().value_counts()
new_tokens = list(vc[vc>100].index)
d2 = {token : i+len(tokens) for i,token in enumerate(new_tokens)}

In [9]:
def get_new_token(token):
    if type(token) == int:
        return d[token]
    elif type(token) == str:
        token = token.strip()
        if token in d2:
            return d2[token]
        else:
            return 0
    else:
        raise

In [14]:
tqdm.pandas()
tokenized_messages2 = tokenized_messages.progress_apply(lambda a: [get_new_token(i) for i in a])

/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [19]:
embedding_weights_word2vec = torch.tensor(nlp.vocab.vectors.data[tokens])
embedding_weights_new = torch.randn(len(d2)+3,300)
embedding_weights = torch.cat([embedding_weights_word2vec,embedding_weights_new])

In [23]:
tokenized_messages2.to_pickle("saved/word2vec/tokenized_messagess2.pickle")

In [4]:
tokenized_messages2 = pd.read_pickle("saved/word2vec/tokenized_messagess2.pickle")

In [21]:
torch.save(embedding_weights, "saved/word2vec/word2vec_weights_big")

In [19]:
df2 = df2.copy()

In [9]:
df2['tokenized_message'] = tokenized_messages2

In [10]:
df2.to_pickle("saved/word2vec/df_word2vec_tokenized_big.pickle")

# Next

In [6]:
embedding_weights = torch.load("saved/word2vec/word2vec_weights_big")

SEP_TOKEN = embedding_weights.shape[0]-1
START_TOKEN = embedding_weights.shape[0]-2
END_TOKEN = embedding_weights.shape[0]-3

del embedding_weights

In [3]:
df3 = pd.read_pickle("saved/word2vec/df_word2vec_tokenized_big.pickle")

In [4]:
tqdm.pandas()
df3['token_len'] = df3.tokenized_message.progress_apply(len)
d_interaction_id = {j:i for i,j in enumerate(df3.interaction.unique())}

tqdm.pandas()
df3['speaker_mask'] = df3.progress_apply(lambda a:[d_interaction_id[a.interaction]]*(a.token_len + 1), axis=1)

/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
df3.to_pickle("saved/word2vec/df_word2vec_tokenized_big.pickle")

In [2]:
df3 = pd.read_pickle("saved/word2vec/df_word2vec_tokenized_big.pickle")

In [3]:
def join_lists(*lists, sep):
    lists = lists[0].values
    def _f(*lists, sep):
        for l in lists[:-1]:
            yield from l
            if sep is not None:
                yield sep
        yield from lists[-1]
    return list(_f(*lists,sep=sep))

In [4]:
from functools import partial

In [7]:
j = partial(join_lists, sep=SEP_TOKEN)
conversations_tokenized = df3.groupby('conversation_id').tokenized_message.agg(j)
tqdm.pandas()
conversations_tokenized = conversations_tokenized.progress_apply(lambda a: [START_TOKEN, *a, END_TOKEN] )

/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
df3.drop(['actor_id','timestamp','status','delivery_error'],axis=1, inplace=True)

In [11]:
j_speaker = partial(join_lists, sep=None)
speaker_mask = df3.groupby('conversation_id').speaker_mask.agg(j_speaker)
tqdm.pandas()
speaker_mask = speaker_mask.progress_apply(lambda a: [*a, 0] )

/home/daniel.cahn/anaconda3/envs/msc/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [12]:
word2vec_df = conversations_tokenized.to_frame().join(speaker_mask)

In [13]:
word2vec_df.to_pickle("saved/word2vec/word2vec_df_big.pickle")